# Querying Data from `FTA Bus Award StoryMap`

Currently FTA uses ArcGIS storymap to display their bus award data. Is is possible to extract this data using their REST server.
Extracting bus award data from a server will be the most reliable way to see the data.

In [1]:
import geopandas as gpd
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

In [2]:
from calitp_data_analysis.sql import to_snakecase
import _bus_cost_utils

# Get geojson from ArcGIS Rest Server 

https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query
https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query


# Steps to Query Rest Server

1. Set `Where:` field to `1=1`
2. Set `Out Field:` field to `*`
3. Set `Format:` to `Geojson`
4. Click `Query(GET)`
5. Copy paste given URL 

In [3]:
fy24 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

fy23 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=" 


fy_24_data = to_snakecase(gpd.read_file(fy24))
fy_23_data = to_snakecase(gpd.read_file(fy23))

display(
    fy_24_data.shape,
    type(fy_24_data),
    fy_24_data.info(),
    fy_23_data.shape,
    type(fy_23_data),
    fy_23_data.info(),
)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   application_number_       117 non-null    object  
 1   agency_name               117 non-null    object  
 2   low_no_bbf                117 non-null    object  
 3   project_description       117 non-null    object  
 4   number_of_buses_          117 non-null    int64   
 5   funding_amount            116 non-null    float64 
 6   project_title             117 non-null    object  
 7   project_type              117 non-null    object  
 8   display_type              117 non-null    object  
 9   overall_propulsion_type   117 non-null    object  
 10  specific_propulsion_type  117 non-null    object  
 11  areas_served              117 non-null    object  
 12  congressional_districts   117 non-null    object  
 13  address_line_1            117 non-null    

(117, 22)

geopandas.geodataframe.GeoDataFrame

None

(130, 25)

geopandas.geodataframe.GeoDataFrame

None

# Explore FY23 data

In [4]:
fy_23_data = fy_23_data.assign(
    specific_propulsion_type = fy_23_data["project_description"].apply(_bus_cost_utils.new_prop_finder)
)

In [5]:
fy_23_data.columns

Index(['application_number', 'fta_region', 'project_sponsor', 'project_title',
       'project_description', 'original_statement_of_work', 'funding',
       'number_of_buses', 'traditional_buses', 'low_emission_buses',
       'zero_emission_buses', 'project_type', 'display_type', 'areas_served',
       'congressional_districts', 'propulsion_type', 'address1', 'address2',
       'city', 'state', 'zip', 'x', 'y', 'objectid', 'geometry',
       'specific_propulsion_type'],
      dtype='object')

In [6]:
col_list =[
    "application_number",
    "project_sponsor",
    "project_description",
    "original_statement_of_work",
    "project_type",
    "funding",
    "number_of_buses",
    "zero_emission_buses",
    "propulsion_type",
    "specific_propulsion_type"
]

fy_23_zeb_bus = fy_23_data[
    (fy_23_data["project_type"] == "Bus") &
    (fy_23_data["propulsion_type"] == "Zero")
    #(fy_23_data["low_emission_buses"] == 0) &
    #(fy_23_data["zero_emission_buses"] > 0)
][col_list]

In [7]:
fy_23_data["project_sponsor"].value_counts().head()

New Mexico Department of Transportation on behalf of North Central Regional Transit District    2
State of California on behalf of Kern Regional Transit                                          2
Regional Public Transportation Authority (Valley Metro)                                         1
Metropolitan Transit Authority of Harris County (METRO)                                         1
State of California on behalf of Glenn County Transportation Commission                         1
Name: project_sponsor, dtype: int64

In [8]:
fy_23_data["propulsion_type"].value_counts()

Zero                        54
Low                         43
Traditional                 17
Zero / Traditional           5
Zero / Low                   3
Low / Traditional            3
Other                        2
Zero / Low / Traditional     2
Combined                     1
Name: propulsion_type, dtype: int64

In [9]:
display(fy_23_zeb_bus.shape,
        fy_23_zeb_bus.columns
       )

(4, 10)

Index(['application_number', 'project_sponsor', 'project_description',
       'original_statement_of_work', 'project_type', 'funding',
       'number_of_buses', 'zero_emission_buses', 'propulsion_type',
       'specific_propulsion_type'],
      dtype='object')

In [10]:
# looks pretty good. will have to use some of the functions identify specific prop type and if projects include non-bus items
fy_23_zeb_bus.head(2)

,application_number,project_sponsor,project_description,original_statement_of_work,project_type,funding,number_of_buses,zero_emission_buses,propulsion_type,specific_propulsion_type
49,LOWNO-2023-3705,Comanche Nation,Comanche Nation Transit will receive funds to buy battery-electric vehicles and charging equipment and train workers in the new technology. The no-emission vehicles will help improve air quality and fill transportation gaps for residents of Kiowa Comanche Apache tribal lands.,"Comanche Nation Transit is submitting a proposal for the Low-No Emissions Grant. Funding requested is $300,000. The funding will go towards purchasing no emissions vehicles as well as purchasing the infrastructure to support the vehicles. There will also be a workforce development plan to prepare our staff and offer them more opportunity to expand as employees.",Bus,"$300,000",2,2,Zero,BEB
112,LowNo-2023-4146,King County Metro Transit,"King County Metro Transit will receive funding to buy battery-electric buses and charging equipment and train workers to maintain the electric fleet. The project will convert 27 bus routes that serve areas with higher numbers of people of low income as well as expand Metro’s apprenticeship program, including promoting transit careers for residents in underserved communities.","Purchase thirty 40-foot battery electric buses (BEBs) to replace thirty 60 hybrid diesel/electric buses that are beyond their FTA defined useful life benchmark. The buses would support converting 27 routes that operate out of Metros South Base Campus to full zero emissions operations. These routes serve areas which have a higher proportion of Black , Indigenous, and People of Color, and low-income populations than the rest of the County. See Attachment 1b.The project also supports Metros Workforce and Pathways Development programs including: 1) Expands Metros 48-month registered apprenticeship programs (four new apprentices). 2) Expands supportive services to help purchase tools and provide dependent care (ten apprentices). 3) Provides inclusive community outreach events to promote career pathways targeting residents in underserved communities. 4) Includes train-the-trainer classes (15) for Metros Battery Electric Bus Training Academy to support advanced workforce training.",Bus,"$33,552,634",30 (BEBs),30,Zero,BEB


# Explore FY 24 data

In [11]:
list(fy_24_data.columns)

['application_number_',
 'agency_name',
 'low_no_bbf',
 'project_description',
 'number_of_buses_',
 'funding_amount',
 'project_title',
 'project_type',
 'display_type',
 'overall_propulsion_type',
 'specific_propulsion_type',
 'areas_served',
 'congressional_districts',
 'address_line_1',
 'city',
 'state',
 'additional_zip',
 'zip_code',
 'x',
 'y',
 'fid2',
 'geometry']

In [12]:
fy_24_data.head(2)


,application_number_,agency_name,low_no_bbf,project_description,number_of_buses_,funding_amount,project_title,project_type,display_type,overall_propulsion_type,specific_propulsion_type,areas_served,congressional_districts,address_line_1,city,state,additional_zip,zip_code,x,y,fid2,geometry
0,BUSFAC-2024-4750,"Alaska Department of Transportation, on behalf of City and Borough of Juneau, Capital Transit",BBF,"Alaska Department of Transportation, on behalf of Capital Transit, will receive funding to buy new electric buses, install charging equipment, and initiate a workforce development program. The project will improve service, reliability and air quality throughout the city and borough of Juneau in Alaska’s Southeast panhandle.",6,11855112.0,Mitigating Overcrowding and Enhancing Sustainability in Alaskas Capital City,Vehicle | Facility | Chargers,Zero Bus and Facility,Zero,Battery electric,Rural,AK-001,PO Box 112500,Juneau,AK,2500,99811,-134.41630,58.29971,1,POINT (-134.41631 58.29971)
1,BUSFAC-2024-5061,City of Montgomery / The M Public Transit System,BBF,"The city of Montgomery's The M transit system will receive funding to buy battery electric buses, charging equipment and to initiate a workforce training program. The project will ensure continued service reliability, maintain a state of good repair and, by improving air quality, advance environmental justice in the Montgomery community.",8,16941377.0,City of Montgomery / The M Public Transit System Electric Bus Program,Vehicle | Chargers,Zero Bus and Facility,Zero,Battery electric,Large Urban,AL-002 | AL-007,103 N. Perry Street,Montgomery,AL,3728,36104,-86.30811,32.37930,2,POINT (-86.30811 32.37930)


In [13]:
display(
    list(fy_24_data["specific_propulsion_type"].sort_values().unique()),
    list(fy_24_data["project_type"].sort_values().unique())
)


['Battery electric',
 'Battery electric ',
 'Battery electric | Diesel-electric hybrid',
 'Battery electric | Diesel-electric hybrid | Propane',
 'Battery electric | Hydrogen fuel cell',
 'Battery electric | Propane',
 'CNG',
 'CNG | Diesel | Diesel-electric hybrid | Gasoline | Other',
 'CNG | Diesel-electric hybrid',
 'CNG | Gasoline',
 'CNG | Hydrogen fuel cell',
 'Diesel',
 'Diesel | Gasoline',
 'Diesel-electric hybrid',
 'Diesel-electric hybrid | Hydrogen fuel cell',
 'Gasoline',
 'Hydrogen fuel cell',
 'N/A',
 'Propane']

['Facility',
 'Facility ',
 'Facility | Chargers',
 'Facility | Equipment',
 'Vehicle',
 'Vehicle ',
 'Vehicle | Chargers',
 'Vehicle | Chargers | Equipment',
 'Vehicle | Equipment',
 'Vehicle | Facility',
 'Vehicle | Facility ',
 'Vehicle | Facility | Chargers',
 'Vehicle | Facility | Equipment']

In [14]:
col_list_24 = [
    "application_number_",
    "agency_name",
    "project_description",
    "project_type",
    "funding_amount",
    "number_of_buses_",
    "overall_propulsion_type",
    "specific_propulsion_type"
]

search_val=[
    "Vehicle ", 
    "Vehicle"
]

prop_vals = [
    "Battery electric",
    "Hydrogen fuel cell",
    "Battery electric | Hydrogen fuel cell",
    "Battery electric ",
]

fy_24_zeb_bus = fy_24_data[col_list_24][
    (fy_24_data["project_type"].isin(search_val)) & 
    (fy_24_data["specific_propulsion_type"].isin(prop_vals))
]

# Filtered Dataframes
Filtered to show only ZEB projects for 23 and 24 only. 
Project type is just buses, not facilities, equipment or any combinations including buses

In [15]:
fy_24_zeb_bus

,application_number_,agency_name,project_description,project_type,funding_amount,number_of_buses_,overall_propulsion_type,specific_propulsion_type
8,BUSFAC-2024-4738,"California Department of Transportation, on behalf of Morongo Basin Transit Authority","The California Department of Transportation, on behalf of the Morongo Basin Transit Authority, will receive funding to buy a battery-electric bus to replace an older vehicle. This project will improve air quality, safety, reliability and state of good repair in the San Bernadino County service area.",Vehicle,131168.0,1,Zero,Battery electric
9,BUSFAC-2024-4564,Santa Barbara Metropolitan Transit District,"The Santa Barbara Metropolitan Transportation District will receive funding to buy battery-electric buses and chargers to replace older diesel buses, which will improve service reliability and reduce air pollution.",Vehicle,2894131.0,2,Zero,Battery electric
31,LOWNO-2024-4943,Escambia County Board of County Commissioners,"Escambia County Area Transit (ECAT) will receive funding to buy battery electric buses and install a charging station, part of ECAT's plan to transition to a fully zero-emission fleet by 2035. The agency will prioritize deployment to areas with a higher percentage of underrepresented populations that rely heavily on transit and have been disproportionately impacted by pollution.",Vehicle,21272962.0,13,Zero,Battery electric
49,LOWNO-2024-4643,Greater Peoria Mass Transit District,"Greater Peoria Mass Transit District will receive funding to buy battery-electric buses to replace diesel-engine vehicles. The project will lower emissions, increase safety and provide more reliable transportation to underserved communities in Peoria and surrounding areas.",Vehicle,14415095.0,10,Zero,Battery electric
54,BUSFAC-2024-5032,Transit Authority of River City (TARC),"Transit Authority of River City (TARC) will receive funding to buy zero-emission battery-electric buses to replace buses that have exceeded their useful life. TARC will improve its fleet's state of good repair and provide more environmentally friendly transportation for riders to jobs, school, healthcare and more.",Vehicle,3643825.0,3,Zero,Battery electric
57,LOWNO-2024-4949,Massachusetts Bay Transportation Authority,The Massachusetts Bay Transportation Authority will receive funding to buy new battery electric buses to replace older buses that have exceeded their useful life and additional funds to train workers on the new technology. The zero-emission buses will significantly reduce greenhouse gas emissions and harmful pollutants.,Vehicle,40000000.0,33,Zero,Battery electric
95,LOWNO-2024-4952,Tri-County Metropolitan Transportation District of Oregon (TriMet),"TriMet will receive funding to buy new hydrogen fuel cell buses to replace older diesel buses that are past their useful life, a new portable fueling station and to initiate facility improvements to accommodate the new hydrogen vehicles. The project will improve service, reliability and provide essential connections to jobs in services in the Portland metro area.",Vehicle,39000000.0,14,Zero,Hydrogen fuel cell
106,LOWNO-2024-4980,Utah Transit Authority,"Utah Transit Authority will receive funding to replace older diesel buses with new battery-electric buses, which will be housed at their Meadowbrook facility. This project will reduce harmful emissions, while serving disadvantaged communities within Salt Lake County.",Vehicle,18112632.0,15,Zero,Battery electric
114,LOWNO-2024-4946,Chelan Douglas Public Transportation Benefit Area (dba Link Transit),"Link Transit will receive funding to buy battery electric buses, replacing gas-powered buses that have exceeded their useful life. The new buses will be more reliable and easier to maintain, lower fueling costs, reduce air pollution, and improve service throughout North Central Washington.",Vehicle,4462500.0,5,Zero,Battery electric


In [16]:
# new specific prop type col isnt catching the prop type correctly from 
fy_23_zeb_bus

,application_number,project_sponsor,project_description,original_statement_of_work,project_type,funding,number_of_buses,zero_emission_buses,propulsion_type,specific_propulsion_type
49,LOWNO-2023-3705,Comanche Nation,Comanche Nation Transit will receive funds to buy battery-electric vehicles and charging equipment and train workers in the new technology. The no-emission vehicles will help improve air quality and fill transportation gaps for residents of Kiowa Comanche Apache tribal lands.,"Comanche Nation Transit is submitting a proposal for the Low-No Emissions Grant. Funding requested is $300,000. The funding will go towards purchasing no emissions vehicles as well as purchasing the infrastructure to support the vehicles. There will also be a workforce development plan to prepare our staff and offer them more opportunity to expand as employees.",Bus,"$300,000",2,2,Zero,BEB
112,LowNo-2023-4146,King County Metro Transit,"King County Metro Transit will receive funding to buy battery-electric buses and charging equipment and train workers to maintain the electric fleet. The project will convert 27 bus routes that serve areas with higher numbers of people of low income as well as expand Metro’s apprenticeship program, including promoting transit careers for residents in underserved communities.","Purchase thirty 40-foot battery electric buses (BEBs) to replace thirty 60 hybrid diesel/electric buses that are beyond their FTA defined useful life benchmark. The buses would support converting 27 routes that operate out of Metros South Base Campus to full zero emissions operations. These routes serve areas which have a higher proportion of Black , Indigenous, and People of Color, and low-income populations than the rest of the County. See Attachment 1b.The project also supports Metros Workforce and Pathways Development programs including: 1) Expands Metros 48-month registered apprenticeship programs (four new apprentices). 2) Expands supportive services to help purchase tools and provide dependent care (ten apprentices). 3) Provides inclusive community outreach events to promote career pathways targeting residents in underserved communities. 4) Includes train-the-trainer classes (15) for Metros Battery Electric Bus Training Academy to support advanced workforce training.",Bus,"$33,552,634",30 (BEBs),30,Zero,BEB
123,LowNo-2023-4175,AutoRidad Metropolitana De Autobuses (PRMBA),"The Metropolitan Bus Authority will receive funding to buy new battery-electric buses, which they will maintain using a solar power charging station that was previously supported with federal funds. The buses will operate on high-visibility routes, reduce greenhouse gas emissions, improve reliability and lessen dependence on external energy sources.","PRIME+ proposes to purchase eight no emission buses to eight replace diesel buses and use an FTA FY 2022 awarded dual solar powered charging station to bring sustainable climate friendly transit to Puerto Rico thus effectuating reliable, efficient and clean transportation, as well as, the use of plentiful solar energy. The buses will be assigned to mainline routes showcasing the project to citizens and visitors. PRIME will reduce: fossil fuel consumption, harmful emissions, dependence on foreign energy sources.",Bus,"$10,000,000",8 (zero emission buses),8,Zero,BEB
128,LowNo-2023-4222,North County Transit District (NCTD),"The North County Transit District will receive funding to buy hydrogen fuel-cell electric buses to replace older buses and to develop a workforce development program in partnership with a local college. This project will help reduce greenhouse gas emissions and improve air quality, health, and transit operations in northern San Diego County.","The Accelerate Clean Transit (ACT) Project will deploy 23 hydrogen fuel-cell electric buses (FCEB) and create an Advanced Transportation apprenticeship program in partnership with Palomar College. The project will replace 23 end-of-life compressed natural gas (